In [2]:
import pandas as pd

# caminho do arquivo
 
file_path = 'spotify_dataset.csv'
spotify_df = pd.read_csv(file_path)

transposed_df = spotify_df.head(1).transpose()
# Visualização das primeiras linhas do dataframe
transposed_df


,0
Nome da Faixa,Bulldog Down in Sunny Tennessee
Cantor ou Compositor,Charlie Poole
Genero,Country; International
Quadrante,Q3
acousticness,0.96
danceability,0.649
energy,0.39
instrumentalness,0.000003
key,5
liveness,0.096


In [2]:
# Verificando valores ausentes

missing_values = spotify_df.isnull().sum()
print("Valores Ausentes:\n", missing_values)

# Verificando os tipos de dados

print("\nTipos de Dados:\n", spotify_df.dtypes)


Valores Ausentes:
 Nome da Faixa           0
Cantor ou Compositor    0
Genero                  0
Quadrante               0
acousticness            0
danceability            0
energy                  0
instrumentalness        0
key                     0
liveness                0
loudness                0
mode                    0
speechiness             0
tempo                   0
time_signature          0
valence                 0
dtype: int64

Tipos de Dados:
 Nome da Faixa            object
Cantor ou Compositor     object
Genero                   object
Quadrante                object
acousticness            float64
danceability            float64
energy                  float64
instrumentalness        float64
key                       int64
liveness                float64
loudness                float64
mode                      int64
speechiness             float64
tempo                   float64
time_signature            int64
valence                 float64
dtype: object


In [ ]:
from sklearn.preprocessing import StandardScaler

# Selecionando as colunas numéricas para normalização
num_cols = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']

# Aplicando a normalização
scaler = StandardScaler()
spotify_df[num_cols] = scaler.fit_transform(spotify_df[num_cols])


In [ ]:
from sklearn.model_selection import train_test_split

X = spotify_df[num_cols]  # Características de áudio 
y = spotify_df['Quadrante']  # Variável Target

# Dividindo os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Podemos observar se há alguma oportunidade aqui.


In [ ]:
# Balanceamento do conjunto de treino, teste e validação para que contenham a mesma proporção

from sklearn.model_selection import train_test_split

X = spotify_df[num_cols]  # Características de áudio
y = spotify_df['Quadrante']  # Variável Target

# Dividindo os dados com estratificação
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# divisão em treino e teste temporários
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# divisão do conjunto de treino em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)


# Contagem da variável alvo 'Quadrante' nos conjuntos de treino, teste e validação

print("Distribuição de classes no conjunto de treino:\n", y_train.value_counts(normalize=True))
print("\nDistribuição de classes no conjunto de teste:\n", y_test.value_counts(normalize=True))
print("\nDistribuição de classes no conjunto de validação:\n", y_val.value_counts(normalize=True))


#### KNN V2

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

# Listas para armazenar os resultados
neighbors_settings = range(1, 11)  # Testar de 1 a 10 vizinhos
accuracy_scores = []

# Loop para treinar e testar o KNN com diferentes números de vizinhos
for n_neighbors in neighbors_settings:
    
    # Criar e treinar o modelo
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    
    # Fazer previsões no conjunto de teste
    y_pred = knn.predict(X_test)
    
    # Calcular e armazenar a acurácia
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
    
    # Imprimir o relatório de classificação para cada modelo
    print(f'Relatório de classificação para k={n_neighbors}:')
    print(classification_report(y_test, y_pred), '\n')

# Plotar os resultados
plt.plot(neighbors_settings, accuracy_scores, label='Acurácia do Teste')
plt.ylabel('Acurácia')
plt.xlabel('Número de Vizinhos')
plt.title('KNN: Acurácia vs. Número de Vizinhos')
plt.legend()
plt.show()


In [ ]:
import joblib

# Identificar o melhor número de vizinhos
best_k = neighbors_settings[accuracy_scores.index(max(accuracy_scores))]

# Treinar o modelo novamente com o melhor k
best_knn_model = KNeighborsClassifier(n_neighbors=best_k)
best_knn_model.fit(X_train, y_train)

# Salvar o modelo
joblib.dump(best_knn_model, r'best_knn_model.joblib')

Variando L1 e L2

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Listas para armazenar os resultados
neighbors_settings = range(1, 11)  # Testar de 1 a 10 vizinhos
accuracy_scores_l1 = []
accuracy_scores_l2 = []

# Loop para treinar e testar o KNN com diferentes números de vizinhos e distâncias L1 e L2
for n_neighbors in neighbors_settings:
    
    # KNN com distância L1 (Manhattan)
    knn_l1 = KNeighborsClassifier(n_neighbors=n_neighbors, p=1)
    knn_l1.fit(X_train, y_train)
    y_pred_l1 = knn_l1.predict(X_test)
    accuracy_scores_l1.append(accuracy_score(y_test, y_pred_l1))
    
    # KNN com distância L2 (Euclidiana)
    knn_l2 = KNeighborsClassifier(n_neighbors=n_neighbors, p=2)
    knn_l2.fit(X_train, y_train)
    y_pred_l2 = knn_l2.predict(X_test)
    accuracy_scores_l2.append(accuracy_score(y_test, y_pred_l2))

# Plotar os resultados
plt.figure(figsize=(10, 6))
plt.plot(neighbors_settings, accuracy_scores_l1, label='L1 - Manhattan', marker='o')
plt.plot(neighbors_settings, accuracy_scores_l2, label='L2 - Euclidiana', marker='o')
plt.title('KNN: Acurácia vs. Número de Vizinhos com L1 e L2')
plt.xlabel('Número de Vizinhos')
plt.ylabel('Acurácia')
plt.legend()
plt.show()


In [ ]:
# Considerando que o melhor modelo é um Manhattan com 10 vizinhos, vamos exportá-lo

import joblib

melhor_knn_model = KNeighborsClassifier(n_neighbors=10, p=1)
melhor_knn_model.fit(X_train, y_train)

# avaliar o modelo



# Salvar o modelo

joblib.dump(melhor_knn_model, 'melhor_knn_model.joblib')


#### NAIVE BAYES - V2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score


# Hiperparâmetros para testar
alphas = np.logspace(-2, 2, 10)  # Para MultinomialNB e BernoulliNB
var_smoothing_values = np.logspace(-9, -6, 10)  # Para GaussianNB

# Listas para armazenar os resultados
accuracy_scores_gaussian = []
accuracy_scores_bernoulli = []

# Testar GaussianNB
for var_smoothing in var_smoothing_values:
    model = GaussianNB(var_smoothing=var_smoothing)
    model.fit(X_train, y_train)
    accuracy_scores_gaussian.append(accuracy_score(y_test, model.predict(X_test)))

# Testar BernoulliNB
for alpha in alphas:
    model = BernoulliNB(alpha=alpha)
    model.fit(X_train, y_train)
    accuracy_scores_bernoulli.append(accuracy_score(y_test, model.predict(X_test)))

# Plotar os resultados para GaussianNB
plt.figure(figsize=(14, 6))
plt.subplot(1, 3, 1)
plt.plot(var_smoothing_values, accuracy_scores_gaussian, marker='o')
plt.title('GaussianNB: Acurácia vs Var Smoothing')
plt.xlabel('Var Smoothing')
plt.ylabel('Acurácia')
plt.xscale('log')

# Plotar os resultados para BernoulliNB
plt.subplot(1, 3, 3)
plt.plot(alphas, accuracy_scores_bernoulli, marker='o')
plt.title('BernoulliNB: Acurácia vs Alpha')
plt.xlabel('Alpha')
plt.xscale('log')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import joblib


# Inicializando o GaussianNB com o var_smoothing escolhido considerando o ultimo gráfico
var_smoothing_optimal = 1e-9  # Exemplo de valor, ajuste conforme necessário
best_gaussian_model = GaussianNB(var_smoothing=var_smoothing_optimal)

# Treino do modelo com os dados de treinamento
best_gaussian_model.fit(X_train, y_train)

# previsões no conjunto de teste
y_pred = best_gaussian_model.predict(X_test)

# Calculo da acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do GaussianNB com var_smoothing de 1e-9: {accuracy:.4f}')


# Salve o modelo treinado em um arquivo
joblib.dump(best_gaussian_model, 'best_gaussian_model.joblib')




#### RANDOM FOREST - V2

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Definir o espaço de hiperparâmetros para testar 10 combinações
param_grid = {
    'n_estimators': [10, 50, 100, 150, 200, 250, 300, 350, 400, 450],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [10, 20, 30, 40, 50, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Criar o objeto GridSearchCV
rf_grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    verbose=2,
    scoring='accuracy'
)

# Treinar o modelo com os dados de treinamento e diferentes hiperparâmetros
# Este processo pode levar algum tempo dependendo do tamanho do dataset e do número de combinações
rf_grid_search.fit(X_train, y_train)

# Resultados em um DataFrame para facilitar a visualização e análise
results_df = pd.DataFrame(rf_grid_search.cv_results_)

# Melhores parâmetros e melhor score
best_params = rf_grid_search.best_params_
best_score = rf_grid_search.best_score_

# Salvar o modelo (não será executado aqui devido à limitação do ambiente)
# joblib.dump(rf_grid_search.best_estimator_, 'best_random_forest_model.joblib')

# Extraindo apenas os resultados para os hiperparâmetros mais relevantes
# e criando gráficos baseados nesses resultados

# Filtrando os resultados para n_estimators e max_depth, que são dois dos hiperparâmetros mais impactantes
estimators_results = results_df.loc[results_df['param_max_features'] == 'auto']
depth_results = results_df.loc[results_df['param_max_features'] == 'auto']

# Plotando acurácia vs n_estimators
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.lineplot(data=estimators_results, x='param_n_estimators', y='mean_test_score')
plt.title('Random Forest: Acurácia vs Número de Estimadores')
plt.xlabel('Número de Estimadores')
plt.ylabel('Acurácia Média de Teste')

# Plotando acurácia vs max_depth
plt.subplot(1, 2, 2)
sns.lineplot(data=depth_results, x='param_max_depth', y='mean_test_score')
plt.title('Random Forest: Acurácia vs Profundidade Máxima')
plt.xlabel('Profundidade Máxima')
plt.ylabel('Acurácia Média de Teste')

plt.tight_layout()
plt.show()



#### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import joblib

# Configurar o modelo KNN
knn = KNeighborsClassifier(n_neighbors=5)

# Treinar o modelo
knn.fit(X_train, y_train)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Fazer previsões no conjunto de teste
y_pred = knn.predict(X_test)

# Avaliar o modelo
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))
print("Acurácia: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))

# Salvar o modelo

joblib.dump(knn, 'KNN_MODEL.pkl')


In [ ]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Intervalo de valores para n_neighbors
neighbors = range(1, 26)

# Lista para armazenar as acurácias
accuracies = []

# Testar o modelo com diferentes valores de n_neighbors
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))

plt.figure(figsize=(12, 6))
plt.plot(neighbors, accuracies, color='blue', marker='o', linestyle='dashed', linewidth=2, markersize=10)
plt.title('Acurácia vs. Número de Vizinhos')
plt.xlabel('Número de Vizinhos')
plt.ylabel('Acurácia')
plt.grid(True)
plt.show()

#### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Criar uma instância do Gaussian Naive Bayes
nb = GaussianNB()

# Treinar o modelo
nb.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_nb = nb.predict(X_test)

# Avaliar o modelo
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred_nb))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred_nb))
print("Acurácia: {:.2f}%".format(accuracy_score(y_test, y_pred_nb) * 100))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Obter a matriz de confusão
cm = confusion_matrix(y_test, y_pred_nb)

# Plotar a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')
plt.title('Matriz de Confusão do Naive Bayes')
plt.ylabel('Classe Verdadeira')
plt.xlabel('Classe Predita')
plt.show()

In [ ]:
import numpy as np

# Obter probabilidades preditas
probabilidades = nb.predict_proba(X_test)

# Para simplificar, vamos plotar as probabilidades de uma classe
classe = 0  # ajuste conforme necessário
prob_classe = probabilidades[:, classe]

# Plotar as probabilidades
plt.figure(figsize=(10, 6))
plt.hist(prob_classe, bins=10, color='skyblue', edgecolor='black')
plt.title(f'Probabilidades Preditas para a Classe {classe}')
plt.xlabel('Probabilidade')
plt.ylabel('Frequência')
plt.show()

In [ ]:
# Obter probabilidades preditas
probabilidades = nb.predict_proba(X_test)

# Número de classes
n_classes = probabilidades.shape[1]

plt.figure(figsize=(12, 8))

# Criar um histograma para cada classe
for i in range(n_classes):
    plt.hist(probabilidades[:, i], bins=20, alpha=0.5, label=f'Classe {i}')

plt.title('Probabilidades Preditas para Todas as Classes')
plt.xlabel('Probabilidade')
plt.ylabel('Frequência')
plt.legend(loc='upper center')
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns

# Converter as probabilidades em um DataFrame para facilitar a plotagem
df_prob = pd.DataFrame(probabilidades, columns=[f'Classe {i}' for i in range(n_classes)])

plt.figure(figsize=(12, 8))

# Criar um gráfico de violino para cada classe
sns.violinplot(data=df_prob)
plt.title('Distribuição das Probabilidades Preditas para Todas as Classes')
plt.ylabel('Probabilidade')
plt.show()

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Criar uma instância do modelo Random Forest
rf = RandomForestClassifier(random_state=42)

# Treinar o modelo
rf.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_rf = rf.predict(X_test)

# Avaliar o modelo
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred_rf))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred_rf))
print("Acurácia: {:.2f}%".format(accuracy_score(y_test, y_pred_rf) * 100))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Obter a matriz de confusão
cm_rf = confusion_matrix(y_test, y_pred_rf)

# Plotar a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm_rf, annot=True, fmt="d", cmap='Blues')
plt.title('Matriz de Confusão do Random Forest')
plt.ylabel('Classe Verdadeira')
plt.xlabel('Classe Predita')
plt.show()


In [ ]:
# Obter a importância das características
feature_importances = rf.feature_importances_

# Converter as importâncias em um DataFrame para facilitar a visualização
features_df = pd.DataFrame({'Característica': X_train.columns, 'Importância': feature_importances})

# Ordenar as características pela importância
features_df = features_df.sort_values(by='Importância', ascending=False)

# Plotar a importância das características
plt.figure(figsize=(12, 8))
sns.barplot(x='Importância', y='Característica', data=features_df)
plt.title('Importância das Características no Modelo Random Forest')
plt.xlabel('Importância')
plt.ylabel('Característica')
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Definir as combinações de hiperparâmetros para testar
param_combinations = [
    {'n_estimators': 10, 'max_depth': None, 'min_samples_split': 2},
    {'n_estimators': 50, 'max_depth': 10, 'min_samples_split': 2},
    {'n_estimators': 100, 'max_depth': 20, 'min_samples_split': 4},
    {'n_estimators': 150, 'max_depth': 30, 'min_samples_split': 6},
    {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 8}
]

# Lista para armazenar os resultados
results = []

# Testar cada combinação de hiperparâmetros
for params in param_combinations:
    rf = RandomForestClassifier(n_estimators=params['n_estimators'],
                                max_depth=params['max_depth'],
                                min_samples_split=params['min_samples_split'],
                                random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results.append({'params': params, 'accuracy': accuracy})

# Converter resultados em um DataFrame
results_df = pd.DataFrame(results)


In [ ]:
# Plotar os resultados
plt.figure(figsize=(12, 8))
sns.barplot(x='accuracy', y=results_df['params'].astype(str), data=results_df)
plt.title('Acurácia do Random Forest para Diferentes Combinações de Hiperparâmetros')
plt.xlabel('Acurácia')
plt.ylabel('Combinações de Hiperparâmetros')
plt.show()


#### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Criar uma instância do modelo Gradient Boosting
gbc = GradientBoostingClassifier(random_state=42)

# Treinar o modelo
gbc.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_gbc = gbc.predict(X_test)

# Avaliar o modelo
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred_gbc))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred_gbc))
print("Acurácia: {:.2f}%".format(accuracy_score(y_test, y_pred_gbc) * 100))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Definir as combinações de hiperparâmetros para testar
param_combinations = [
    {'n_estimators': 50, 'learning_rate': 0.01, 'max_depth': 3},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 3},
    {'n_estimators': 150, 'learning_rate': 0.1, 'max_depth': 4},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 5},
    {'n_estimators': 250, 'learning_rate': 0.2, 'max_depth': 4}
]

# Lista para armazenar os resultados
results = []

# Testar cada combinação de hiperparâmetros
for params in param_combinations:
    gbc = GradientBoostingClassifier(n_estimators=params['n_estimators'],
                                     learning_rate=params['learning_rate'],
                                     max_depth=params['max_depth'],
                                     random_state=42)
    gbc.fit(X_train, y_train)
    y_pred = gbc.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results.append({'params': params, 'accuracy': accuracy})

# Converter resultados em um DataFrame
results_df = pd.DataFrame(results)

In [ ]:
# Plotar os resultados
plt.figure(figsize=(12, 8))
sns.barplot(x='accuracy', y=results_df['params'].astype(str), data=results_df)
plt.title('Acurácia do Gradient Boosting para Diferentes Combinações de Hiperparâmetros')
plt.xlabel('Acurácia')
plt.ylabel('Combinações de Hiperparâmetros')
plt.show()


#### XGBoosting

In [ ]:
import xgboost as xgb

from sklearn.preprocessing import LabelEncoder

# Inicializar o LabelEncoder
le = LabelEncoder()

# Ajustar e transformar as classes
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)



# Criar uma instância do modelo XGBoost
xgb_clf = xgb.XGBClassifier(random_state=42)

# Treinar o modelo XGBoost com as classes codificadas
xgb_clf.fit(X_train, y_train_encoded)

# Fazer previsões no conjunto de teste
y_pred_xgb = xgb_clf.predict(X_test)

# Avaliar o modelo (lembre-se de decodificar as previsões de volta para as classes originais se necessário)
print("Matriz de Confusão:\n", confusion_matrix(y_test_encoded, y_pred_xgb))
print("\nRelatório de Classificação:\n", classification_report(y_test_encoded, y_pred_xgb))
print("Acurácia: {:.2f}%".format(accuracy_score(y_test_encoded, y_pred_xgb) * 100))


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Definir as combinações de hiperparâmetros para testar
param_combinations = [
    {'n_estimators': 50, 'learning_rate': 0.01, 'max_depth': 3},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 4},
    {'n_estimators': 150, 'learning_rate': 0.1, 'max_depth': 5},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 6},
    {'n_estimators': 250, 'learning_rate': 0.2, 'max_depth': 7}
]

# Lista para armazenar os resultados
results = []

# Testar cada combinação de hiperparâmetros
for params in param_combinations:
    xgb_model = XGBClassifier(n_estimators=params['n_estimators'],
                              learning_rate=params['learning_rate'],
                              max_depth=params['max_depth'],
                              random_state=42)
    xgb_model.fit(X_train, y_train_encoded)
    y_pred = xgb_model.predict(X_test)
    accuracy = accuracy_score(y_test_encoded, y_pred)
    results.append({'params': params, 'accuracy': accuracy})

# Converter resultados em um DataFrame
results_df = pd.DataFrame(results)

In [ ]:
# Plotar os resultados
plt.figure(figsize=(12, 8))
sns.barplot(x='accuracy', y=results_df['params'].astype(str), data=results_df)
plt.title('Acurácia do XGBoost para Diferentes Combinações de Hiperparâmetros')
plt.xlabel('Acurácia')
plt.ylabel('Combinações de Hiperparâmetros')
plt.show()


### Redes Neurais ####

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder

# Inicializar o LabelEncoder
le = LabelEncoder()

# Ajustar e transformar as classes
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Criar o modelo
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))  # Usando 4 porque temos 4 classes (Q1, Q2, Q3, Q4)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train_encoded, epochs=50, batch_size=32, validation_split=0.2)

# Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)
print("Acurácia: {:.2f}%".format(accuracy * 100))



In [ ]:

import matplotlib.pyplot as plt

# Plotar a acurácia do treinamento e validação
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Acurácia (treinamento)')
plt.plot(history.history['val_accuracy'], label='Acurácia (validação)')
plt.title('Acurácia ao Longo das Épocas')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend()

# Plotar a perda do treinamento e validação
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Perda (treinamento)')
plt.plot(history.history['val_loss'], label='Perda (validação)')
plt.title('Perda ao Longo das Épocas')
plt.ylabel('Perda')
plt.xlabel('Época')
plt.legend()

plt.show()


#### RNA - Mais profunda ####

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# Criar o modelo
model_complex = Sequential()

# Primeira camada oculta
model_complex.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model_complex.add(BatchNormalization())
model_complex.add(Dropout(0.5))

# Segunda camada oculta
model_complex.add(Dense(64, activation='relu'))
model_complex.add(BatchNormalization())
model_complex.add(Dropout(0.5))

# Terceira camada oculta
model_complex.add(Dense(32, activation='relu'))
model_complex.add(BatchNormalization())
model_complex.add(Dropout(0.5))

# Camada de saída
model_complex.add(Dense(4, activation='softmax'))  # 4 classes de saída


In [ ]:
model_complex.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

history_complex = model_complex.fit(X_train, y_train_encoded, epochs=50, batch_size=32, validation_split=0.2)

# Avaliar o modelo
loss, accuracy = model_complex.evaluate(X_test, y_test_encoded, verbose=0)
print("Acurácia: {:.2f}%".format(accuracy * 100))

In [ ]:
import matplotlib.pyplot as plt

# Plotar a acurácia do treinamento e validação
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history_complex.history['accuracy'], label='Acurácia (treinamento)')
plt.plot(history_complex.history['val_accuracy'], label='Acurácia (validação)')
plt.title('Acurácia ao Longo das Épocas')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend()

# Plotar a perda do treinamento e validação
plt.subplot(1, 2, 2)
plt.plot(history_complex.history['loss'], label='Perda (treinamento)')
plt.plot(history_complex.history['val_loss'], label='Perda (validação)')
plt.title('Perda ao Longo das Épocas')
plt.ylabel('Perda')
plt.xlabel('Época')
plt.legend()

plt.show()


#### SVM ####

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# É comum usar pipelines para combinar a etapa de pré-processamento com o modelo
svm_pipeline = make_pipeline(StandardScaler(), SVC(probability=True))

# Treinar o modelo com os dados de treinamento
svm_pipeline.fit(X_train, y_train)

# Previsões no conjunto de Teste

y_pred_svm = svm_pipeline.predict(X_test)

# Avaliação do modelo

print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred_svm))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred_svm))
print("Acurácia: {:.2f}%".format(accuracy_score(y_test, y_pred_svm) * 100))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Definir o espaço de hiperparâmetros para testar
param_grid = {
    'C': [0.1, 1, 10, 100, 1000],             # Regularização
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],   # Parâmetro do Kernel
    'kernel': ['rbf']                          # Tipo do Kernel
}

# Criar o objeto GridSearchCV
grid_search = GridSearchCV(SVC(), param_grid, refit=True, verbose=3, cv=5)

# Treinar o modelo SVM com os dados de treinamento e diferentes hiperparâmetros
grid_search.fit(X_train, y_train)

# Melhores parâmetros e melhor score
print("Melhores Parâmetros:", grid_search.best_params_)
print("Melhor Score:", grid_search.best_score_)


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Reshape the mean test scores into a 5x5 grid for the values of 'C' and 'gamma'
scores = results['mean_test_score'].reshape(5, 5)

# Plot a heatmap
plt.figure(figsize=(8, 8))
sns.heatmap(scores, annot=True, xticklabels=param_grid['gamma'], yticklabels=param_grid['C'], cmap='viridis')
plt.xlabel('Gamma')
plt.ylabel('C')
plt.title('Acurácia do GridSearchCV para o SVM')
plt.show()

In [ ]:
import joblib


# Acessar o melhor estimador encontrado pelo GridSearchCV
best_svm_model = grid_search.best_estimator_

# Salvar o melhor modelo
joblib.dump(best_svm_model, 'best_svm_model.pkl')
